In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import cv2
import os
import torch
from tqdm.notebook import tqdm

In [2]:
df_masks = pd.read_csv('train.csv')

In [52]:
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    s = encs.split()
    
    for i in range(len(s)//2):
        start = int(s[2*i]) - 1
        length = int(s[2*i+1])
        img[start:start+length] = 1
        
    return img.reshape(shape).T

In [72]:
from datetime import datetime

In [77]:
date_now = datetime.now()

In [86]:
reduce = 1  #тесты с уменьшением размерности
MASKS = '../input/hubmap-kidney-segmentation/train.csv'
DATA = 'train/'
OUTPUT = f'tiles_ready/tiles_{date_now.month}m_{date_now.day}d_{date_now.hour}h'

In [91]:
hyper_params = {
    'tile_size':1024,
    'stride':1024
}

In [92]:
import os

In [95]:
os.makedirs(f'{OUTPUT}/imgs', exist_ok=True)
os.makedirs(f'{OUTPUT}/masks', exist_ok=True)

In [101]:
s_th = 40  #saturation blancking threshold
p_th = 200*sz//256 #threshold for the minimum number of pixels

x_tot, x2_tot = [], []

img_patches = []
mask_patches = []

for _, row in tqdm(df_masks.iterrows(), total=len(df_masks)):
    index, encs = row
    
    img = tiff.imread(os.path.join(DATA, index+'.tiff'))
    if len(img.shape) == 5: img = np.transpose(img.squeeze(), (1,2,0))
    mask = enc2mask(encs,(img.shape[1],img.shape[0]))

    shape = img.shape
    pad0 = (reduce*sz - shape[0]%(reduce*sz))%(reduce*sz)
    pad1 = (reduce*sz - shape[1]%(reduce*sz))%(reduce*sz)
        
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=0)
    mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2]],
                constant_values=0)
        
    img = torch.tensor(img).unfold(0, sz, stride).unfold(1, sz, stride)
    mask = torch.tensor(mask).unfold(0, sz, stride).unfold(1, sz, stride)

    img = img.reshape(img.shape[0]*img.shape[1], *img.shape[2:])
    mask = mask.reshape(mask.shape[0]*mask.shape[1], *mask.shape[2:])
    
    img = img.detach().numpy()
    mask = mask.detach().numpy()
    
    img = np.transpose(img, (0, 2, 3, 1))
        
    for i,(im,m) in enumerate(zip(img,mask)):                
        hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if (s>s_th).sum() <= p_th or im.sum() <= p_th: continue

#         x_tot.append((im/255.0).reshape(-1,3).mean(0))
#         x2_tot.append(((im/255.0)**2).reshape(-1,3).mean(0))
#         plt.imshow(im)
#         break
#         im = cv2.imencode('.png',cv2.cvtColor(im, cv2.COLOR_RGB2BGR))[1]
        
#         print(f'{OUTPUT}/imgs/{index}_{i}.png', f'{OUTPUT}/masks/{index}_{i}.png')

        cv2.imwrite(f'{OUTPUT}/imgs/{index}_{i}.png', im)
        
        cv2.imwrite(f'{OUTPUT}/masks/{index}_{i}.png', m)

  0%|          | 0/8 [00:00<?, ?it/s]